In [2]:

import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from photutils.psf import fit_fwhm
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from astropy.stats import sigma_clip
import os

# List of all reduced image paths
reduced_image_paths = [

    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\M52\B-band\M52_normalized_Stacked_B-band.fits', 'M52_B'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\M52\U-band\M52_normalized_Stacked_U-band.fits', 'M52_U'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\M52\V-band\M52_normalized_Stacked_V-band.fits', 'M52_V'),


        (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\NGC7789\B-band\NGC7789_normalized_Stacked_B-band.fits', 'NGC7789_B'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\NGC7789\U-band\NGC7789_normalized_Stacked_U-band.fits', 'NGC7789_U'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\NGC7789\V-band\NGC7789_normalized_Stacked_V-band.fits', 'NGC7789_V'),

    # Standard Star 1 - B-band
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 1\B-band\First Observation\SS1 normalized_Stacked_First Observation B.fits', 'Standard_Star_1_B_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 1\B-band\Second Observation\SS1 normalized_Stacked_Second Observation B.fits', 'Standard_Star_1_B_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 1\B-band\Third Observation\SS1 normalized_Stacked_Third Observation B.fits', 'Standard_Star_1_B_3rd'),

    # Standard Star 1 - U-band
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 1\U-band\First Observation\SS1 normalized_Stacked_First Observation U.fits', 'Standard_Star_1_U_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 1\U-band\Second Observation\SS1 normalized_Stacked_Second Observation U.fits', 'Standard_Star_1_U_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 1\U-band\Third Observation\SS1 normalized_Stacked_Third Observation U.fits', 'Standard_Star_1_U_3rd'),

    # Standard Star 1 - V-band
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 1\V-band\First Observation\SS1 normalized_Stacked_First Observation V.fits', 'Standard_Star_1_V_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 1\V-band\Second Observation\SS1 normalized_Stacked_Second Observation V.fits', 'Standard_Star_1_V_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 1\V-band\Third Observation\SS1 normalized_Stacked_Third Observation V.fits', 'Standard_Star_1_V_3rd'),

    # # Standard Star 2 - B-band
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 2\B-band\First Observation\SS2 normalized_Stacked_First Observation B.fits', 'Standard_Star_2_B_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 2\B-band\Second Observation\SS2 normalized_Stacked_Second Observation B.fits', 'Standard_Star_2_B_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 2\B-band\Third Observation\SS2 normalized_Stacked_Third Observation B.fits', 'Standard_Star_2_B_3rd'),

    # # Standard Star 2 - U-band
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 2\U-band\First Observation\SS2 normalized_Stacked_First Observation U.fits', 'Standard_Star_2_U_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 2\U-band\Second Observation\SS2 normalized_Stacked_Second Observation U.fits', 'Standard_Star_2_U_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 2\U-band\Third Observation\SS2 normalized_Stacked_Third Observation U.fits', 'Standard_Star_2_U_3rd'),

    # # Standard Star 2 - V-band
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 2\V-band\First Observation\SS2 normalized_Stacked_First Observation V.fits', 'Standard_Star_2_V_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 2\V-band\Second Observation\SS2 normalized_Stacked_Second Observation V.fits', 'Standard_Star_2_V_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized Aligned Stacked Images\Standard Star 2\V-band\Third Observation\SS2 normalized_Stacked_Third Observation V.fits', 'Standard_Star_2_V_3rd'),
]

# Dictionary to store results for each image
results = {}

# Loop through each reduced image path
for image_path, label in reduced_image_paths:
    print(f"Processing: {label}")
    
    # Open the FITS file and load the image data
    if os.path.exists(image_path):
        with fits.open(image_path) as hdul:
            data = hdul[0].data
            mean, median, std = sigma_clipped_stats(data, sigma=3.0)

            # Detect stars and calculate FWHM
            daofind = DAOStarFinder(fwhm=4.0, threshold=7.0 * std)
            sources = daofind(data - median)

            if sources is None or len(sources) == 0:
                print(f"No sources found for {label}.")
                continue

            sources.sort('flux', reverse=True)
            sources = sources[:500]  # Adjust the number 100 as needed for your analysis

            # Get positions of detected stars
            xypos = np.transpose((sources['xcentroid'], sources['ycentroid']))

            try:
                # Calculate FWHM for detected stars and ensure fit_shape is odd
                median_fwhm = np.median(fit_fwhm(data, xypos=xypos, fit_shape=7))
                fit_shape = int(round(1.5 * median_fwhm))
                if fit_shape % 2 == 0:  # Ensure fit_shape is odd
                    fit_shape += 1

                fwhm_values = fit_fwhm(data, xypos=xypos, fit_shape=fit_shape)
                fwhm_values_clipped = sigma_clip(fwhm_values, sigma=3.0, maxiters=5)
                median_fwhm_clipped = np.median(fwhm_values_clipped[~fwhm_values_clipped.mask])
                print(f"{label} - Clipped Median FWHM: {median_fwhm_clipped:.2f} pixels")

                # Set up aperture and annulus sizes based on median FWHM
                aperture_radius = 3.0 * median_fwhm_clipped
                inner_radius = 2.0 * aperture_radius
                outer_radius = 3.0 * aperture_radius

                # Define the aperture and annulus objects
                apertures = CircularAperture(xypos, r=aperture_radius)
                annulus_apertures = CircularAnnulus(xypos, r_in=inner_radius, r_out=outer_radius)

                # Check that n_sky > n_pix
                n_sky = annulus_apertures.area
                n_pix = apertures.area
                if n_sky <= n_pix:
                    print(f"Warning: Increase annulus size for {label} to ensure n_sky > n_pix.")

                # Perform aperture photometry on the data
                phot_table = aperture_photometry(data, apertures)
                bkg_table = aperture_photometry(data, annulus_apertures)

                # Calculate background-subtracted flux
                bkg_mean = bkg_table['aperture_sum'] / annulus_apertures.area
                phot_table['residual_aperture_sum'] = phot_table['aperture_sum'] - (bkg_mean * apertures.area)

                # Filter for positive residual_aperture_sum to avoid log errors
                positive_flux = phot_table['residual_aperture_sum'] > 0
                phot_table = phot_table[positive_flux]
                
                # Calculate instrumental magnitude for positive flux values
                phot_table['instrumental_mag'] = -2.5 * np.log10(phot_table['residual_aperture_sum'])

                # Store results in the dictionary
                results[label] = phot_table[['id', 'xcenter', 'ycenter', 'residual_aperture_sum', 'instrumental_mag']]
                print(f"Photometry results for {label} stored.")

            except Exception as e:
                print(f"Error processing {label}: {e}")


for label, result in results.items(): # Display all results after processing
    print(f"\nResults for {label}:")
    print(result)


Processing: Standard_Star_2_U_1st


c:\Users\bewan\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


Standard_Star_2_U_1st - Clipped Median FWHM: 0.45 pixels
Photometry results for Standard_Star_2_U_1st stored.
Processing: Standard_Star_2_U_2nd


c:\Users\bewan\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


Standard_Star_2_U_2nd - Clipped Median FWHM: 1.16 pixels
Photometry results for Standard_Star_2_U_2nd stored.
Processing: Standard_Star_2_U_3rd


Standard_Star_2_U_3rd - Clipped Median FWHM: 1.02 pixels
Photometry results for Standard_Star_2_U_3rd stored.

Results for Standard_Star_2_U_1st:
 id      xcenter       ... residual_aperture_sum   instrumental_mag 
           pix         ...                                          
--- ------------------ ... --------------------- -------------------
  1 31.741137153630767 ...    184.85277247897818   -5.66706492160189
  2 2066.5112241363972 ...     173.0082457045782  -5.595167006068495
  3 109.55263044598084 ...    125.90292111338832  -5.250089516070066
  4  2648.750529470963 ...     75.20548920281651  -4.690623851595141
  5  3713.983545294355 ...     72.89366733721086  -4.656724501180202
  6  690.2494231546292 ...      62.8162860043895  -4.495180638199765
  7 3775.6753074466146 ...    53.413084396843686  -4.319119143755083
  8  2119.363355042782 ...     50.28005583603562  -4.253489378246251
  9  850.7049899163104 ...     45.35401757129867  -4.141539409954652
...                ... ...

c:\Users\bewan\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [ ]:
# # Retrieve data for a specific image
# m52_b_data = results.get('M52_B')
# print("Photometry results for M52_B:")
# print(m52_b_data)


In [3]:
import pandas as pd

# Save each set of results to a CSV file
for label, photometry_data in results.items():
    try:
        # Convert photometry data (Astropy Table) to a Pandas DataFrame
        df = photometry_data.to_pandas()

        # Define the CSV filename using the label
        csv_filename = f"{label}_photometry_results.csv"

        # Save the DataFrame to a CSV file
        df.to_csv(csv_filename, index=False)

        print(f"Saved photometry results for {label} to {csv_filename}")
    except Exception as e:
        print(f"Error saving results for {label}: {e}")


Saved photometry results for Standard_Star_2_U_1st to Standard_Star_2_U_1st_photometry_results.csv
Saved photometry results for Standard_Star_2_U_2nd to Standard_Star_2_U_2nd_photometry_results.csv
Saved photometry results for Standard_Star_2_U_3rd to Standard_Star_2_U_3rd_photometry_results.csv


C:\Users\bewan\AppData\Local\Temp\ipykernel_25504\3188057790.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
#Below is code for without alligning the images

In [1]:

'''
THE ONE TRUE CODE
'''

import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from photutils.psf import fit_fwhm
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from astropy.stats import sigma_clip
import os

# List of all reduced image paths
reduced_image_paths = [
    # Standard Star 1 - B-band
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS1\B-band\1st\normalized_SS1_Stacked_1st_B.fits', 'Standard_Star_1_B_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS1\B-band\2nd\normalized_SS1_Stacked_2nd_B.fits', 'Standard_Star_1_B_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS1\B-band\3rd\normalized_SS1_Stacked_3rd_B.fits', 'Standard_Star_1_B_3rd'),

    # Standard Star 1 - U-band
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS1\U-band\1st\normalized_SS1_Stacked_1st_U.fits', 'Standard_Star_1_U_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS1\U-band\2nd\normalized_SS1_Stacked_2nd_U.fits', 'Standard_Star_1_U_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS1\U-band\3rd\normalized_SS1_Stacked_3rd_U.fits', 'Standard_Star_1_U_3rd'),

    # Standard Star 1 - V-band
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS1\V-band\1st\normalized_SS1_Stacked_1st_V.fits', 'Standard_Star_1_V_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS1\V-band\2nd\normalized_SS1_Stacked_2nd_V.fits', 'Standard_Star_1_V_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS1\V-band\3rd\normalized_SS1_Stacked_3rd_V.fits', 'Standard_Star_1_V_3rd'),

    # Standard Star 2 - B-band
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS2\B-band\1st\normalized_SS2_Stacked_1st_B.fits', 'Standard_Star_2_B_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS2\B-band\2nd\normalized_SS2_Stacked_2nd_B.fits', 'Standard_Star_2_B_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS2\B-band\3rd\normalized_SS2_Stacked_3rd_B.fits', 'Standard_Star_2_B_3rd'),

    # Standard Star 2 - U-band
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS2\U-band\1st\normalized_SS2_Stacked_1st_U.fits', 'Standard_Star_2_U_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS2\U-band\2nd\normalized_SS2_Stacked_2nd_U.fits', 'Standard_Star_2_U_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS2\U-band\3rd\normalized_SS2_Stacked_3rd_U.fits', 'Standard_Star_2_U_3rd'),

    # Standard Star 2 - V-band
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS2\V-band\1st\normalized_SS2_Stacked_1st_V.fits', 'Standard_Star_2_V_1st'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS2\V-band\2nd\normalized_SS2_Stacked_2nd_V.fits', 'Standard_Star_2_V_2nd'),
    (r'G:\MyProject\TGP\data_reduction\Normalized without Alligning\SS2\V-band\3rd\normalized_SS2_Stacked_3rd_V.fits', 'Standard_Star_2_V_3rd'),
]



# Dictionary to store results for each image
results = {}

# Loop through each reduced image path
for image_path, label in reduced_image_paths:
    print(f"Processing: {label}")
    
    # Open the FITS file and load the image data
    if os.path.exists(image_path):
        with fits.open(image_path) as hdul:
            data = hdul[0].data
            mean, median, std = sigma_clipped_stats(data, sigma=3.0)

            # Step 1: Detect stars and calculate FWHM
            daofind = DAOStarFinder(fwhm=4.0, threshold=7.0 * std)
            sources = daofind(data - median)

            if sources is None or len(sources) == 0:
                print(f"No sources found for {label}.")
                continue

            sources.sort('flux', reverse=True)
            sources = sources[:500]  # Adjust the number 100 as needed for your analysis

            # Get positions of detected stars
            xypos = np.transpose((sources['xcentroid'], sources['ycentroid']))

            try:
                # Calculate FWHM for detected stars and ensure fit_shape is odd
                median_fwhm = np.median(fit_fwhm(data, xypos=xypos, fit_shape=7))
                fit_shape = int(round(1.5 * median_fwhm))
                if fit_shape % 2 == 0:  # Ensure fit_shape is odd
                    fit_shape += 1

                fwhm_values = fit_fwhm(data, xypos=xypos, fit_shape=fit_shape)
                fwhm_values_clipped = sigma_clip(fwhm_values, sigma=3.0, maxiters=5)
                median_fwhm_clipped = np.median(fwhm_values_clipped[~fwhm_values_clipped.mask])
                print(f"{label} - Clipped Median FWHM: {median_fwhm_clipped:.2f} pixels")

                # Step 2: Set up aperture and annulus sizes based on median FWHM
                aperture_radius = 3.0 * median_fwhm_clipped
                inner_radius = 2.0 * aperture_radius
                outer_radius = 3.0 * aperture_radius

                # Define aperture and annulus objects
                apertures = CircularAperture(xypos, r=aperture_radius)
                annulus_apertures = CircularAnnulus(xypos, r_in=inner_radius, r_out=outer_radius)

                # Check that n_sky > n_pix
                n_sky = annulus_apertures.area
                n_pix = apertures.area
                if n_sky <= n_pix:
                    print(f"Warning: Increase annulus size for {label} to ensure n_sky > n_pix.")

                # Step 3: Perform aperture photometry
                phot_table = aperture_photometry(data, apertures)
                bkg_table = aperture_photometry(data, annulus_apertures)

                # Calculate background-subtracted flux
                bkg_mean = bkg_table['aperture_sum'] / annulus_apertures.area
                phot_table['residual_aperture_sum'] = phot_table['aperture_sum'] - (bkg_mean * apertures.area)

                # Filter for positive residual_aperture_sum to avoid log errors
                positive_flux = phot_table['residual_aperture_sum'] > 0
                phot_table = phot_table[positive_flux]
                
                # Calculate instrumental magnitude for positive flux values
                phot_table['instrumental_mag'] = -2.5 * np.log10(phot_table['residual_aperture_sum'])

                # Store results in dictionary
                results[label] = phot_table[['id', 'xcenter', 'ycenter', 'residual_aperture_sum', 'instrumental_mag']]
                print(f"Photometry results for {label} stored.")

            except Exception as e:
                print(f"Error processing {label}: {e}")

# Display all results after processing
for label, result in results.items():
    print(f"\nResults for {label}:")
    print(result)


Processing: Standard_Star_1_B_1st


c:\Users\bewan\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


Standard_Star_1_B_1st - Clipped Median FWHM: 3.45 pixels
Photometry results for Standard_Star_1_B_1st stored.
Processing: Standard_Star_1_B_2nd


c:\Users\bewan\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


Standard_Star_1_B_2nd - Clipped Median FWHM: 3.98 pixels
Photometry results for Standard_Star_1_B_2nd stored.
Processing: Standard_Star_1_B_3rd


...

c:\Users\bewan\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


Standard_Star_2_V_1st - Clipped Median FWHM: 4.76 pixels
Photometry results for Standard_Star_2_V_1st stored.
Processing: Standard_Star_2_V_2nd


c:\Users\bewan\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


Standard_Star_2_V_2nd - Clipped Median FWHM: 4.20 pixels
Photometry results for Standard_Star_2_V_2nd stored.
Processing: Standard_Star_2_V_3rd
Standard_Star_2_V_3rd - Clipped Median FWHM: 4.37 pixels
Photometry results for Standard_Star_2_V_3rd stored.

Results for Standard_Star_1_B_1st:
 id      xcenter       ... residual_aperture_sum   instrumental_mag 
           pix         ...                                          
--- ------------------ ... --------------------- -------------------
  1   3384.04249871809 ...     141013.2000385645 -12.873149420567282
  2   3383.28229598163 ...    141028.36305684605 -12.873266162495362
  3  3384.533275245395 ...     140804.4617004946  -12.87154104151105
  4 3383.4199497507916 ...    140911.20823002854 -12.872363846841138
  5 2682.2878108301857 ...     38449.33150599067 -11.462221983504662
  6   887.979879932707 ...    12990.942837325876 -10.284101679489474
  7  955.6848186137079 ...    11583.433416335085 -10.159593266517819
  8  2761.7153302921

c:\Users\bewan\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [2]:
import pandas as pd

# Save each set of results to a CSV file
for label, photometry_data in results.items():
    try:
        # Convert photometry data (Astropy Table) to a Pandas DataFrame
        df = photometry_data.to_pandas()

        # Define the CSV filename using the label
        csv_filename = f"{label}_photometry_results.csv"

        # Save the DataFrame to a CSV file
        df.to_csv(csv_filename, index=False)

        print(f"Saved photometry results for {label} to {csv_filename}")
    except Exception as e:
        print(f"Error saving results for {label}: {e}")


Saved photometry results for Standard_Star_1_B_1st to Standard_Star_1_B_1st_photometry_results.csv
Saved photometry results for Standard_Star_1_B_2nd to Standard_Star_1_B_2nd_photometry_results.csv
Saved photometry results for Standard_Star_1_B_3rd to Standard_Star_1_B_3rd_photometry_results.csv
Saved photometry results for Standard_Star_1_U_1st to Standard_Star_1_U_1st_photometry_results.csv
Saved photometry results for Standard_Star_1_U_2nd to Standard_Star_1_U_2nd_photometry_results.csv
Saved photometry results for Standard_Star_1_V_1st to Standard_Star_1_V_1st_photometry_results.csv
Saved photometry results for Standard_Star_1_V_2nd to Standard_Star_1_V_2nd_photometry_results.csv
Saved photometry results for Standard_Star_2_B_1st to Standard_Star_2_B_1st_photometry_results.csv
Saved photometry results for Standard_Star_2_B_2nd to Standard_Star_2_B_2nd_photometry_results.csv
Saved photometry results for Standard_Star_2_B_3rd to Standard_Star_2_B_3rd_photometry_results.csv
Saved phot

C:\Users\bewan\AppData\Local\Temp\ipykernel_30900\3188057790.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
